### DATA SOURCES FOR HCBB PROJECT

- Physician & Other Supplier Payments: https://www.cms.gov/Research-Statistics-Data-and-Systems/Statistics-Trends-and-Reports/Medicare-Provider-Charge-Data/Physician-and-Other-Supplier2017, Detailed Data

- Hospital Outpatient: https://www.cms.gov/Research-Statistics-Data-and-Systems/Statistics-Trends-and-Reports/Medicare-Provider-Charge-Data/Outpatient, Detailed Data, APC to CPT/HCPCS crosswalk

- https://www.cms.gov/Medicare/Medicare-Fee-for-Service-Payment/HospitalOutpatientPPS/Addendum-A-and-Addendum-B-Updates, Addendum B – January 2020 (correction files aren't necessary), 

- Zip Code to CBSA: https://www.huduser.gov/portal/datasets/usps_crosswalk.html

- Other: Data.CMS.gov

In [7]:
import pandas as pd
import pickle
import xlrd

## READ IN BY CHUNKS THE HOSPITAL PROVIDER FILE (LARGE)

In [9]:
%%time

outpatient_df = pd.DataFrame()
chunk_nbr = 1

for chunk in pd.read_csv('../class_data/Provider_Outpatient_Hospital_Charge_Data_by_APC__CY2017.csv', chunksize = 1000):
    if chunk_nbr == 1:
        outpatient_df = outpatient_df.append(chunk)
        chunk_nbr += 1
    else: break

print(outpatient_df.shape)
outpatient_df.head()

Wall time: 43.9 ms


,Provider_ID,Provider_Name,Provider_Street_Address,Provider_City,Provider_State,Provider_Zip_Code,Provider_HRR,APC,APC_Desc,Beneficiaries,CAPC_Services,Average_Total_Submitted_Charges,Average_Medicare_Allowed_Amount,Average_Medicare_Payment_Amount,Outlier_Services,Average_Medicare_Outlier_Amount
0,10001,Southeast Alabama Medical Center,1108 Ross Clark Circle,Dothan,AL,36301,AL - Dothan,5072,Level 2 Excision/ Biopsy/ Incision and Drainage,249.0,259,9575.01,1038.45,826.28,NaN,NaN
1,10001,Southeast Alabama Medical Center,1108 Ross Clark Circle,Dothan,AL,36301,AL - Dothan,5073,Level 3 Excision/ Biopsy/ Incision and Drainage,52.0,53,12578.28,1792.60,1423.25,NaN,NaN
2,10001,Southeast Alabama Medical Center,1108 Ross Clark Circle,Dothan,AL,36301,AL - Dothan,5091,Level 1 Breast/Lymphatic Surgery and Related P...,26.0,27,11337.61,2113.58,1683.99,0.0,0.0
3,10001,Southeast Alabama Medical Center,1108 Ross Clark Circle,Dothan,AL,36301,AL - Dothan,5092,Level 2 Breast/Lymphatic Surgery and Related P...,23.0,23,17116.16,3737.14,2977.55,0.0,0.0
4,10001,Southeast Alabama Medical Center,1108 Ross Clark Circle,Dothan,AL,36301,AL - Dothan,5112,Level 2 Musculoskeletal Procedures,17.0,17,7382.73,1029.46,820.21,0.0,0.0


In [36]:
outpatient_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 16 columns):
Provider_ID                        1000 non-null int64
Provider_Name                      1000 non-null object
Provider_Street_Address            1000 non-null object
Provider_City                      1000 non-null object
Provider_State                     1000 non-null object
Provider_Zip_Code                  1000 non-null int64
Provider_HRR                       1000 non-null object
APC                                1000 non-null int64
APC_Desc                           1000 non-null object
Beneficiaries                      989 non-null float64
CAPC_Services                      1000 non-null int64
Average_Total_Submitted_Charges    1000 non-null float64
Average_Medicare_Allowed_Amount    1000 non-null float64
Average_Medicare_Payment_Amount    1000 non-null float64
Outlier_Services                   788 non-null float64
Average_Medicare_Outlier_Amount    788 non-null float64

## READ IN AND CLEAN ADDENDUM FILE

In [21]:
addendum = pd.read_excel('../data/2020_january_web_addendum_b.12312019.xlsx', skiprows=2, header=[0])
addendum = addendum.dropna(subset = ['APC ']).reset_index(drop=True)
addendum = addendum.astype({'APC ': 'str'})
addendum.head(10)

,HCPCS Code,Short Descriptor,SI,APC,Relative Weight,Payment Rate,National Unadjusted Copayment,Minimum Unadjusted Copayment,"Note: Actual copayments would be lower due to the cap on copayments at the Inpatient Deductible of $1,408.00",* Indicates a Change
0,10005,Fna bx w/us gdn 1st les,T,5071.0,7.5503,610.01,.,122.01,NaN,NaN
1,10007,Fna bx w/fluor gdn 1st les,T,5071.0,7.5503,610.01,.,122.01,NaN,NaN
2,10009,Fna bx w/ct gdn 1st les,T,5071.0,7.5503,610.01,.,122.01,NaN,NaN
3,10011,Fna bx w/mr gdn 1st les,T,5071.0,7.5503,610.01,.,122.01,NaN,NaN
4,10021,Fna bx w/o img gdn 1st les,T,5052.0,3.9547,319.51,.,63.91,NaN,NaN
5,10030,Guide cathet fluid drainage,T,5071.0,7.5503,610.01,.,122.01,NaN,NaN
6,10035,Perq dev soft tiss 1st imag,T,5071.0,7.5503,610.01,.,122.01,NaN,NaN
7,10040,Acne surgery,Q1,5051.0,2.1627,174.73,.,34.95,NaN,NaN
8,10060,Drainage of skin abscess,T,5051.0,2.1627,174.73,.,34.95,NaN,NaN
9,10061,Drainage of skin abscess,T,5052.0,3.9547,319.51,.,63.91,NaN,NaN


## READ IN CBSA/ZIP CROSSWALK FILE

In [23]:
zip_crosswalk_df = pd.read_excel('../data/CBSA_ZIP_032020.xlsx')
zip_crosswalk_df.head()

,CBSA,ZIP,RES_RATIO,BUS_RATIO,OTH_RATIO,TOT_RATIO
0,10100,57445,0.044115,0.016156,0.037037,0.041101
1,10100,57456,0.000046,0.000000,0.000000,0.000039
2,10100,57441,0.014690,0.003401,0.000639,0.012785
3,10100,57474,0.005792,0.003401,0.001277,0.005295
4,10100,57401,0.751576,0.788265,0.933589,0.766139
